In [32]:
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam

In [33]:
def preprocess_images(directory, batch_size, target_size, shuffle=True):
    datagen = ImageDataGenerator(
        rescale=1.0/255.0,  
        shear_range=0.2,    
        zoom_range=0.2,     
        horizontal_flip=True,  
        rotation_range=30,   
        width_shift_range=0.2,   
        height_shift_range=0.2  
    )

    image_generator = datagen.flow_from_directory(
        directory,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=shuffle
    )

    return image_generator


In [34]:


# Define the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(15, activation='softmax'))  # Assuming 3 classes


model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 26, 26, 128)     

In [35]:

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [36]:
# Preprocess the image data
train_data = preprocess_images('Vegetable Images/train', batch_size=32, target_size=(224, 224))
test_data = preprocess_images('Vegetable Images/test', batch_size=32, target_size=(224, 224))
val_data = preprocess_images('Vegetable Images/validation', batch_size=32, target_size=(224, 224))



Found 15000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.
Found 3000 images belonging to 15 classes.


In [30]:
class_names=['Bean', 'Bitter_Gourd', 'Bottle_Gourd', 'Brinjal', 'Broccoli', 'Cabbage', 'Capsicum', 'Carrot', 'Cauliflower', 'Cucumber', 'Papaya', 'Potato', 'Pumpkin', 'Radish', 'Tomato']
class_names

['Bean',
 'Bitter_Gourd',
 'Bottle_Gourd',
 'Brinjal',
 'Broccoli',
 'Cabbage',
 'Capsicum',
 'Carrot',
 'Cauliflower',
 'Cucumber',
 'Papaya',
 'Potato',
 'Pumpkin',
 'Radish',
 'Tomato']

In [4]:
with tf.device('GPU:0'):
# Train the model
    model.fit(train_data,
              epochs=10,
              validation_data=val_data)

    # Evaluate the model on the test data
    test_loss, test_accuracy = model.evaluate(test_data)
    print('Test Loss:', test_loss)
    print('Test Accuracy:', test_accuracy)

    

Epoch 1/10
469/469 [==============================] - 577s 1s/step - loss: 1.4173 - accuracy: 0.5223 - val_loss: 1.0775 - val_accuracy: 0.6407
Epoch 2/10
469/469 [==============================] - 546s 1s/step - loss: 0.7412 - accuracy: 0.7595 - val_loss: 0.5293 - val_accuracy: 0.8270
Epoch 3/10
469/469 [==============================] - 321s 684ms/step - loss: 0.5216 - accuracy: 0.8309 - val_loss: 0.4324 - val_accuracy: 0.8617
Epoch 4/10
469/469 [==============================] - 369s 784ms/step - loss: 0.4084 - accuracy: 0.8689 - val_loss: 0.3679 - val_accuracy: 0.8820
Epoch 5/10
469/469 [==============================] - 380s 811ms/step - loss: 0.3358 - accuracy: 0.8953 - val_loss: 0.3410 - val_accuracy: 0.8937
Epoch 6/10
469/469 [==============================] - 418s 891ms/step - loss: 0.3084 - accuracy: 0.9019 - val_loss: 0.3060 - val_accuracy: 0.9050
Epoch 7/10
469/469 [==============================] - 389s 829ms/step - loss: 0.2773 - accuracy: 0.9141 - val_loss: 0.3116 - val_a

In [5]:
model.save('vegetablemodel.h5')

In [37]:
import numpy as np
import cv2
def predict(model, img):
    image = Image.open(img)  
    image = image.resize((224, 224)) 
    image = np.array(image) / 255.0  
    image = np.expand_dims(image, axis=0) 
    predictions = model.predict(image)
    predicted_class = np.argmax(predictions)
    predicted_label = class_names[predicted_class]
    return ("Predicted Label:", predicted_label)


In [40]:
img = '0009.jpg'
model = tf.keras.models.load_model('vegetablemodel.h5')
predict(model,img)

1/1 [==============================] - 1s 517ms/step


('Predicted Label:', 'Bitter_Gourd')

In [51]:
!pip install tf2onnx


     -------------------------------------- 451.2/451.2 kB 1.0 MB/s eta 0:00:00
  Using cached onnx-1.14.0-cp310-cp310-win_amd64.whl (13.3 MB)


In [53]:
import onnx
import tf2onnx

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (C:\Users\Raja.Balasubramani\AppData\Roaming\Python\Python310\site-packages\google\protobuf\internal\__init__.py)

In [ ]:
import tf2